In [1]:
from hermes.triton.client import TritonClient
from hermes.openvino.client import OpenVinoClient
from musicspeech_controller import MusicSpeechController
from musicspeech_params import MusicSpeech_Params as music_speech_params

2024-04-10 11:15:13.971357: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 11:15:14.087829: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-10 11:15:14.087851: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-04-10 11:15:14.612718: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

## Triton

In [2]:
client = TritonClient("music-detection", {"ip": "0.0.0.0", "grpc_port": 8101})

In [3]:
client.metadata

{'name': 'music-detection',
 'versions': ['1'],
 'platform': 'tensorflow_savedmodel',
 'inputs': [{'name': 'mel_input',
   'datatype': 'FP32',
   'shape': ['-1', '802', '80']}],
 'outputs': [{'name': 'time_distributed',
   'datatype': 'FP32',
   'shape': ['-1', '802', '2']}]}

In [4]:
controller = MusicSpeechController(client=client, params=music_speech_params)

2024-04-10 11:15:15.446556: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-04-10 11:15:15.446578: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-10 11:15:15.446597: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (condor2363): /proc/driver/nvidia/version does not exist
2024-04-10 11:15:15.446743: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
input_path = "../Cortes_music_nmusic/139850/139850.ogg"
output_path = "../Cortes_music_nmusic/139850/139850_trecho_preds.txt"

In [ ]:
segments = controller.predict(input_path)

with open(output_path, 'w') as fp:
    fp.write('\n'.join('{}\t{}\t{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in segments))

## OpenVino

In [ ]:
client = OpenVinoClient("music-detection", {"ip": "0.0.0.0", "grpc_port": 8101})

In [ ]:
client.metadata